In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_155686/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_43_0,0.999809,0.823459,0.816240,0.999862,0.000797,1.180533,0.657299,0.000690,0.066319,0.028223,1.012247,0.028668,144.270410,285.605582,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_43_1,0.999804,0.823394,0.816438,0.999838,0.000816,1.180968,0.656592,0.000811,0.073074,0.028559,1.012541,0.029009,144.223073,285.558245,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_42_5,0.999801,0.823865,0.816461,0.999425,0.000829,1.177817,0.656509,0.001763,0.072216,0.028798,1.012751,0.029252,144.189750,285.524922,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_42_9,0.999799,0.823731,0.816004,0.999324,0.000836,1.178709,0.658143,0.002072,0.070263,0.028906,1.012847,0.029361,144.174816,285.509988,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_42_6,0.999799,0.823879,0.816131,0.999358,0.000836,1.177719,0.657690,0.001965,0.071731,0.028913,1.012853,0.029368,144.173883,285.509055,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,model_1_3_2,0.906825,0.720928,0.751299,0.886265,0.387843,1.866158,0.889588,0.306640,0.640292,0.622770,6.963190,0.632578,131.894311,273.229483,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
468,model_1_3_1,0.904787,0.718809,0.749732,0.883550,0.396328,1.880325,0.895194,0.313960,0.635277,0.629546,7.093647,0.639460,131.851028,273.186201,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
469,model_1_3_0,0.902906,0.717329,0.748450,0.881182,0.404155,1.890220,0.899781,0.320344,0.638576,0.635732,7.213989,0.645744,131.811916,273.147088,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
470,model_1_2_1,0.883463,0.702596,0.741144,0.853835,0.485089,1.988742,0.925914,0.715807,0.737690,0.696483,8.458373,0.707452,131.446847,272.782019,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
